In [ ]:
# default_exp client

In [ ]:
#hide
# just removing the insecure warning for now
# TODO: Secure requests and remove this code
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
#hide
from private import server_vars

In [ ]:
#hide
base_url=server_vars['base_url']
login_email = server_vars['login_email']
login_pwd = server_vars['login_pwd']

`<<<<<<< HEAD`

In [ ]:
# export
import requests

from yx_motor.api import API
from yx_motor.authenticate import Authenticate
from yx_motor.jobs import Jobs
from yx_motor.files import Files


class Client:
    "Wrapper for Alteryx Server API."

    def __init__(self, base_url: str, login_email: str, login_pwd: str):
        """Initialize a yx_motor client object."""
        self.base_url = base_url
        self.api_url = f"{base_url}api/v1/"

        self.auth_endpoint = "authenticate"

        self.login_email = login_email
        self.login_pwd = login_pwd

        self.headers = {
            "Content-Type": "application/json",
            "Accept": "*/*",
            "Accept-Language": "en-US,en;q=0.5",
            "Accept-Encoding": "gzip,deflate",
        }

        self.api = API(api_url=self.api_url, headers=self.headers)

        self.authenticate = Authenticate(self.api)
        self.jobs = Jobs(self.api)

        self.authenticate.authenticate(
            login_email=self.login_email, login_pwd=self.login_pwd
        )

    def get_users(self):
        response = self.api.get("users")
        return response

`=======`

In [ ]:
# export
import requests

from yx_motor.api import API
from yx_motor.authenticate import Authenticate
from yx_motor.jobs import Jobs
from yx_motor.files import Files


class Client:
    "Wrapper for Alteryx Server API."
    def __init__(self,
                 base_url: str,
                 login_email: str,
                 login_pwd: str):
        """Initialize a yx_motor client object."""
        self.base_url = base_url
        self.api_url = f"{base_url}api/v1/"

        self.auth_endpoint = "authenticate"

        self.login_email = login_email
        self.login_pwd = login_pwd

        self.headers = {
            "Content-Type": "application/json",
            "Accept": "*/*",
            "Accept-Language": "en-US,en;q=0.5",
            "Accept-Encoding": "gzip,deflate",
        }

        self.api = API(api_url=self.api_url,
                       headers=self.headers)

        self.authenticate = Authenticate(self.api)
        self.jobs = Jobs(self.api)
        self.files = Files(self.api)

        self.authenticate.authenticate(login_email=self.login_email,
                                       login_pwd=self.login_pwd)

    def get_users(self):
        response = self.api.get("users")
        return response

`>>>>>>> 49e5de1647cd0cae4a23329fa30a66b9a4a03698`

In [ ]:
from nbdev.showdoc import *
show_doc(Client.__init__)

<h4 id="Client.__init__" class="doc_header"><code>Client.__init__</code><a href="__main__.py#L13" class="source_link" style="float:right">[source]</a></h4>

> <code>Client.__init__</code>(**`base_url`**:`str`, **`login_email`**:`str`, **`login_pwd`**:`str`)

Initialize a yx_motor client object.

In [ ]:
motor = Client(base_url,
              login_email, 
              login_pwd)

POST sent to: https://test.aep-mono.devops.alteryx.com/api/v1/authenticate/
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Content-Length': '59'}
Response Status: 200


In [ ]:
#hide
from unittest.mock import Mock
motor.api = Mock()
motor.api.is_authenticated = True

In [ ]:
motor.api.is_authenticated

True

# Jesse and JP testing stuff below (remove/breakout later)

## VFS Router API Investigation

In [ ]:
motor = Client(base_url,
              login_email, 
              login_pwd)

In [ ]:
# Gets the payload of recently viewed files for a given user
response = motor.api.get("files/recentlyViewed")

GET sent to: https://test.aep-mono.devops.alteryx.com/api/v1/files/recentlyViewed
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Cookie': 'ayxSession=s%3Aa898447b-b8c3-4f02-a81c-e61893597599.aSf3VFvnXVqnep%2Fr20sP5IL48NVG%2BkuCzb%2Fhsn%2BnCp0'}
Response Status: 200


In [ ]:
response

<Response [200]>

### Recently Viewed Files Response Object

- Keys at top level
 - members  (what is this exactly?  Looks like it has access control stuff for file)
 - totalCount: integer, exactly what it sounds like
 - assets: array of what appears to be vfs "asset/file" objects.  

In [ ]:
response.json().keys()

dict_keys(['members', 'totalCount', 'assets'])

In [ ]:
asset_list = response.json()['assets']

### VFS File object investigation

- fileName: exactly what you think, but no path
- path: full path, weird in a vfs as Jesse has pointed out
- folderType: What is that and what are its implications?
- version: integer
- maxVersion: Latest version.  interesting, should investigate if there are different uuids for different version or same uuid?????
- uuid: very important, core to all things workflow, vfs, etc.  all workflows appear to be vfs assets.  
- metadata: object with critical keys regarding permissions, etc.
- activityTime: ???  last run??
- contentId: Why and what is this?  Is it useful?
- entryOwner:  What is this for?  Why so much?
- permissions: list of strings of file permission types
- assetCategory: string, what are the valid types for this?  differences in behavior???
- onlySiteAdminShares: bool: Per Jesse, this is a setting on a file to ensure only admins can share or not share a file

In [ ]:
test_asset = asset_list[0]

In [ ]:
asset_list

[{'fileName': 'Magic-8-Ball.yxmd',
  'extension': 'yxmd',
  'originalLocation': None,
  'inherits': True,
  'isHidden': False,
  'onlyOwnerShares': True,
  'path': '/Workspaces/Public/Magic-8-Ball.yxmd',
  'folderType': None,
  'version': 1,
  'created': '2020-04-16T20:42:21.558Z',
  'versionCreated': '2020-04-16T20:42:21.558Z',
  'maxVersion': 1,
  'links': None,
  'location': '',
  'uuid': 'c681db11-31fc-4854-8b0f-7e81870f9d87',
  'metadata': {'yxType': 'WORKFLOW',
   'vfsInputs': {},
   'workflowInfo': {'isE2': False, 'modules': ['Magic-8-Ball.yxmd']}},
  'activityTime': '2020-04-16T20:45:19.075Z',
  'contentHash': '07949d23663f98c18f527fd7af961d209cc4459be11bd39df122b10e119981ba',
  'contentId': 'ba8bba3e-92c0-4f60-a612-7ebec39c2f9f',
  'contentSize': 9226,
  'entryOwner': {'avatar': None,
   'email': 'pladmin@example.com',
   'firstName': 'Seeded',
   'id': 'ef376193-fa7d-45fb-a9d3-57e14ec71d38',
   'lastName': 'pladmin',
   'name': 'Seeded pladmin',
   'userName': 'pladmin'},
  '

In [ ]:
test_asset['uuid']

'1a847899-95a9-4206-a354-a69934c30b8f'

In [ ]:
test_asset['version']

1

In [ ]:
test_asset['path']

'/users/siteadmin3/Magic-8-Ball.yxmd'

## Download File

#### File Content

Seems like content isn't what I thought it was.  

EDIT:  No, this works, just not with the file I tried before.  Note to self, make sure to double check GUI in future.

In [ ]:
# Didn't return a file, but not sure.

response = motor.api.get(url='files/content', 
                         params={"id": 'c70447a5-6f4e-4125-bc50-258c5119368b', 
                                 "version": 1})

Below code successfully writes the file in proper format.

In [ ]:
with open('czech_devops.yxmd', 'wb') as f:
    f.write(response.content)

## Upload File

This is pretty weird.  

Talk to Alexander Potanin.  

Currently, this API is incredibly confusing.  It isn't clear, even by observing browser traffic, how a file actually gets uploaded.

Using headers for the parameters is inconsistent with, hmm, i don't know, everything else in here.  

Latest note:

https://git.alteryx.com/s2/demo-seeder/-/blob/dev/04-UploadFilesAndDecorate.js

see that we just need to specify what appears to be a binary object as payload.  

We will need to add to the existing headers for this one, due to the api inconsistencies.

Added an optional arg (just to post) for non_default_headers.

Success!  uploaded a file with the below code.

### TODO:  The code below works, so we will want to convert into a nice clean function.

In [ ]:
blob = None

with open('8_ball_test.yxzp', "rb") as f:
    blob = f.read()

In [ ]:
upload_headers = {"Content-Type": "application/json",
                    "Accept": "*/*",
                    "Accept-Language": "en-US,en;q=0.5",
                    "Accept-Encoding": "gzip,deflate",
                    "path": '/Workspaces/Public/8_ball_test.yxzp'
                  }

In [ ]:
response = motor.api.post(url='files', 
                          data=blob,
                          non_default_headers=upload_headers)

In [ ]:
response.json()

{'fileName': '8_ball_test.yxzp',
 'extension': 'yxzp',
 'inherits': True,
 'isHidden': False,
 'onlyOwnerShares': False,
 'path': '/Workspaces/Public/8_ball_test.yxzp',
 'folderType': None,
 'version': 1,
 'created': '2020-04-13T19:46:29.407Z',
 'versionCreated': '2020-04-13T19:46:29.407Z',
 'maxVersion': 1,
 'links': None,
 'location': '',
 'uuid': '0a607201-5bdd-4942-aef3-16de2f21c47d',
 'metaHash': 'd214d65ab55a2a1c73de7c90fcc1aecfc28dd88dbec71cdeaeb3ffb1ef170053',
 'metadata': {'yxType': 'WORKFLOW_APP'},
 'contentHash': '07949d23663f98c18f527fd7af961d209cc4459be11bd39df122b10e119981ba',
 'contentId': '93b5a0d4-a3cd-4560-aaba-2bdaef3a40db',
 'contentSize': 9226,
 'entryOwner': {'avatar': None,
  'email': 'siteadmin1@example.com',
  'firstName': 'Seeded',
  'id': 'cb5f4bb6-48a4-4796-add8-5dd96feb773a',
  'lastName': 'siteadmin1',
  'name': 'Seeded siteadmin1',
  'userName': 'siteadmin1'},
 'md5Hash': '8984d04bae22a3af18dcdd317055b088',
 'assetCategory': 'WORKFLOW_APP',
 'onlySiteAdmi

In [ ]:
response.json()

{'fileName': 'jp_test.yxzp',
 'extension': 'yxzp',
 'inherits': True,
 'isHidden': False,
 'onlyOwnerShares': False,
 'path': '/Workspaces/Public/asdf/jp_test.yxzp',
 'folderType': None,
 'version': 1,
 'created': '2020-04-10T23:21:20.252Z',
 'versionCreated': '2020-04-10T23:21:20.252Z',
 'maxVersion': 1,
 'links': None,
 'location': '',
 'uuid': 'bee0b8dd-6387-4740-be36-06887e0d9921',
 'metaHash': 'd214d65ab55a2a1c73de7c90fcc1aecfc28dd88dbec71cdeaeb3ffb1ef170053',
 'metadata': {'yxType': 'WORKFLOW_APP'},
 'contentHash': 'b7960c08c75e5e4e2bcf5fa32582eee75a07560554c6993c603759a8a2ce2f5f',
 'contentId': '6a411b4e-6e57-4d27-b357-421e280aaa51',
 'contentSize': 2176,
 'entryOwner': {'avatar': None,
  'email': 'siteadmin1@example.com',
  'firstName': 'Seeded',
  'id': '3ec0bcbd-f7e2-413b-93c0-166ee2aee5d8',
  'lastName': 'siteadmin1',
  'name': 'Seeded siteadmin1',
  'userName': 'siteadmin1'},
 'md5Hash': '1316e622c535fcfef109f75484910f38',
 'assetCategory': 'WORKFLOW_APP',
 'onlySiteAdminSh

In [ ]:
#hide

from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_client.ipynb.
Converted 02_jobs.ipynb.
Converted 03_api.ipynb.
Converted 04_authenticate.ipynb.
Converted index.ipynb.
